In [ ]:
from pathlib import Path

import numba as nb
import numpy as np
import scipy.stats as stats

from natsort import natsorted

import matplotlib.pyplot as plt


#plt.style.use("mike")
import warnings
warnings.simplefilter("ignore")

warnings.filterwarnings('ignore')


import astropy.units as u

import cmasher as cmr

green = "#33FF86"
purple = "#CE33FF"

%matplotlib widget
from cosmogrb.universe.survey import Survey

import os, sys
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from zusammen.stan_models.stan_model import get_model
from zusammen import AnalysisBuilder, DataSet
from zusammen.spectral_plot import display_posterior_model_counts

from threeML import update_logging_level

import arviz as av


update_logging_level("FATAL")


from astromodels import Band_Calderone, PointSource, Model


from threeML import JointLikelihood, DataList, display_spectrum_model_counts

from astromodels import Cutoff_powerlaw

import popsynth as ps


In [ ]:
survey = Survey.from_file('data/survey.h5')
ab = AnalysisBuilder(survey, use_bb=True)

In [ ]:
ab.write_yaml("test_proc.yml")

In [ ]:
ds = DataSet.from_yaml("test_proc.yml")


In [ ]:
ds.to_hdf5_file("sgrb.h5")

In [ ]:
ds.to_hdf5_file("sgrb.h5", sig_threshold=40)

In [ ]:
ds = DataSet.from_hdf5_file('sgrb.h5')

In [ ]:
m = get_model("cpl_simple_chunked")

In [ ]:
m.clean_model()

In [ ]:
m.build_model()

In [ ]:
data = ds.to_stan_dict()

n_threads = 3
n_chains = 2

fit = m.model.sample(
    
       data=data,
    parallel_chains=n_chains,
    chains=n_chains,
    #inits= {'alpha':-1.},
    threads_per_chain=n_threads,
    seed=1234,
    iter_warmup=1000,
    iter_sampling=500,
    max_treedepth=12,
    show_progress=True,)
    
    

In [ ]:
 m.model.sample?

In [ ]:
res = av.from_cmdstanpy(fit)

In [ ]:
res.sample_stats.treedepth.max()

In [ ]:
av.plot_trace(res)

In [ ]:
bc = Band_Calderone(opt=0, a =10, b=1e4)
bc.beta.fix = True

In [ ]:
id = 0

In [ ]:
alpha = res.posterior.alpha.stack(sample=("chain", "draw")).values[id]

ep = res.posterior.log_ec.stack(sample=("chain", "draw")).values[id]

K  = res.posterior.K.stack(sample=("chain", "draw")).values[id]


div = res.sample_stats.diverging.stack(sample=("chain", "draw")).values

samples = np.vstack((K, alpha, 10.**ep))


dl = ds.get_data_list_of_interval(id)

In [ ]:
div.sum()

In [ ]:
# bc = Band_Calderone(a=10.,b=1e4,opt=0)

# bc.beta.fix=True

bc = Cutoff_powerlaw(piv=100)

bc.index.bounds = (None, None)
bc.K.bounds = (None, None)
bc.xc.bounds = (None, None)

model = Model(PointSource("ps",0,0, spectral_shape=bc))

In [ ]:
fig, ax = plt.subplots()


ax.scatter(ep, alpha, alpha=0.1)


In [ ]:
fig, ax = plt.subplots()


ax.scatter(K, alpha, alpha=0.1)

In [ ]:
display_posterior_model_counts(dl[1],
                               model, 
                               samples.T[::20], 
                               min_rate=1e-99,
                               shade=False
                              
                              );

In [ ]:
cpl = Cutoff_powerlaw(piv=100,K=1e-1,xc=200)






dl = ds.get_data_list_of_interval(2)


model = Model(PointSource("ps",0,0, spectral_shape=cpl))


ba = JointLikelihood(model,DataList(*dl))

In [ ]:
ba.fit();

In [ ]:
display_spectrum_model_counts(ba);

In [ ]:
dl[0].view_count_spectrum();

In [ ]:
cpl